# Feed calculator

In [8]:
import pandas as pd
from pulp import *
import numpy as np


# Basic Model

## Process the dataset

In [9]:
df = pd.read_excel("nutrition.xlsx",nrows=64)
cost_df_raw=pd.read_excel('costs.xlsx')
nutrition_df=pd.read_excel('quantity.xlsx')

In [10]:
df.columns = ['ingredient' if col.startswith('INGREDIENT') else col for col in df]

In [11]:
food_items = list(df['ingredient'])
food_items_df=pd.DataFrame(food_items,columns=['food'])

In [12]:
costs_df=food_items_df.merge(cost_df_raw,left_on='food',right_on='Ingredients')
costs_df=costs_df.drop('food',axis=1)
costs={}
for i in range (len(costs_df)):
    costs[costs_df['Ingredients'].iloc[i]]=costs_df['Price'].iloc[i]

In [13]:
#Extract all the nutritions included
nutrition=[]
for i in range (len(nutrition_df)):
    nutrition.append(nutrition_df.iloc[i][0])
#Create dictionay for every single nutrition. The dictionary includes the amount contained in different ingredient.
for i in range(len(nutrition)):
    vars()[nutrition[i]] = dict(zip(food_items,df[nutrition[i]]))
food_vars = LpVariable.dicts("Portion",food_items,lowBound=0,cat='Continuous')

## Add the objective function

In [14]:
prob = LpProblem("Smallholder Layer Starter Diet",LpMinimize)

In [15]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

## Adding constraints

In [16]:
for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
#     print('Min',i)
    
    prob += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
#         print('Max',i)
        prob += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
#     print('------')

In [17]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve(pulp.PULP_CBC_CMD())

1

In [18]:
print("Solution"+"-"*100)
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue,'kg')
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob.objective),2)))

Ingredients=[]
Amount=[]
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        Ingredients.append(v.name[8:])
        Amount.append(v.varValue)
total_cost=round(value(prob.objective),2)
result_df={'Ingredients':Ingredients,'Amount':Amount}
result_df=pd.DataFrame(result_df)
result_df['total_cost']=total_cost

Solution----------------------------------------------------------------------------------------------------
Portion_boneash = 0.0018651074 kg
Portion_caswhole = 0.073674343 kg
Portion_fish = 0.070430241 kg
Portion_maizebranhighq = 0.13354586 kg
Portion_monocapo = 0.00083770807 kg
Portion_rapemeal = 0.20423689 kg
Portion_rice = 0.48865891 kg
Portion_salt = 0.0012428947 kg
Portion_wheat = 0.067520956 kg
The total cost of this balanced diet is: 79.8 in Nigeria Currency 


# Extension 1-1

Question: At most MaxNum of ingredients to use. 

In [19]:
#Interface for selecting the Number of ingredients
MaxNum=3

In [20]:

prob2 = LpProblem("Smallholder Layer Starter Diet",LpMinimize)

food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

# The objective function is added to 'prob' first
prob2 += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
#     print(n)
#     print('Min',i)

    prob2 += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
#         print('Max',i)
        prob2 += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
#     print('------')

for f in food_items:
#     prob2 += food_vars[f]>= food_chosen[f]*0.1
    prob2 += food_vars[f]>= 0

    prob2 += food_vars[f]<= food_chosen[f]*1e5
prob2+= lpSum([food_chosen[f] for f in food_items])<=MaxNum
# The problem is solved using PuLP's choice of Solver
prob2.solve(pulp.PULP_CBC_CMD())

    
print("Solution"+"-"*100)
for v in prob2.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue,'kg')
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob2.objective),2)))

Ingredients=[]
Amount=[]
for v in prob2.variables():
    if v.varValue>0 and v.name[0]=='P':
        Ingredients.append(v.name[8:])
        Amount.append(v.varValue)
total_cost=round(value(prob2.objective),2)
result_df_extension1_1={'Ingredients':Ingredients,'Amount':Amount}
result_df_extension1_1=pd.DataFrame(result_df_extension1_1)
result_df_extension1_1['total_cost']=total_cost

Solution----------------------------------------------------------------------------------------------------
Portion_fish = 0.13110074 kg
Portion_wheat = 0.53863388 kg
Portion_wheatbran = 0.42978379 kg
The total cost of this balanced diet is: 122.11 in Nigeria Currency 


# Extension 1-0

For every ingredient, at lease A certain minimum percentage must be reached. At the same time, it should not exceed the maximum percentage. 

In [21]:
#Interface ===Ashiwanti

In [22]:
ingre_percentage_df=pd.read_excel('ingredients_percentage.xlsx')

In [23]:
ingre_percentage_df.head()

,Afkorting in WEBCALCULATOR,unit,Minimum,Maximum,Rule in webapp
0,barley,%,1,7.0,barley btw 0.01 and 0.07
1,blood,%,1,3.0,blood btw 0.01 and 0.03
2,boneash,%,1,3.0,boneash btw 0.01 and 0.03
3,mbmeal,%,1,4.0,mbmeal btw 0.01 and 0.04
4,cotton,%,1,4.0,cotton btw 0.01 and 0.04


In [24]:
prob3 = LpProblem("Smallholder Layer Starter Diet",LpMinimize)
prob3 += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

for i in range(len(nutrition_df)):
    n = vars()[nutrition_df.iloc[i,0]]
#     print(n)
#     print('Min',i)
    
    prob3 += lpSum([n[f]* food_vars[f] for f in food_items]) >= nutrition_df['Minimum'][i]
    
    if (i in nutrition_df[nutrition_df['Maximum']>0].index):
#         print('Max',i)
        prob3 += lpSum([n[f]* food_vars[f] for f in food_items]) <= nutrition_df['Maximum'][i]
#     print('------')

for f in food_items:
    prob3 += food_vars[f]>= 0
    prob3 += food_vars[f]<= food_chosen[f]*100

    
# prob3+= lpSum([food_chosen[f] for f in food_items])<=6


for i in range(len(ingre_percentage_df)):
    pmin=ingre_percentage_df.iloc[i][2]/100
    x_i=ingre_percentage_df.iloc[i][0]
    prob3 += -100*(1-food_chosen[x_i])+pmin*lpSum([food_vars[f] for f in food_items])<=food_vars[x_i]
    
    if (i in ingre_percentage_df[ingre_percentage_df['Maximum']>0].index):
#         print('Max',i)
        pmax=ingre_percentage_df.iloc[i][3]/100
        prob3 += pmax*lpSum([food_vars[f] for f in food_items])>=food_vars[x_i]
                                                                          

# The problem is solved using PuLP's choice of Solver
prob3.solve(pulp.PULP_CBC_CMD())
print("Solution"+"-"*100)

for v in prob3.variables():
    if v.varValue>0 and v.name[0]=='P':
#     if v.varValue>0 :
        print(v.name, "=", v.varValue,'kg')
print("The total cost of this balanced diet is: {} in Nigeria Currency ".format(round(value(prob3.objective),2)))

Ingredients=[]
Amount=[]
for v in prob3.variables():
    if v.varValue>0 and v.name[0]=='P':
        Ingredients.append(v.name[8:])
        Amount.append(v.varValue)
total_cost=round(value(prob3.objective),2)
result_df_extention1_0={'Ingredients':Ingredients,'Amount':Amount}
result_df_extention1_0=pd.DataFrame(result_df_extention1_0)
result_df_extention1_0['total_cost']=total_cost

Solution----------------------------------------------------------------------------------------------------
Portion_barley = 0.077685928 kg
Portion_blood = 0.01109799 kg
Portion_copra = 0.050178481 kg
Portion_fish = 0.1109799 kg
Portion_maizebranhighq = 0.22195979 kg
Portion_maizebranlowq = 0.1109799 kg
Portion_palmoil = 0.033649752 kg
Portion_rapecake = 0.044391959 kg
Portion_rapemeal = 0.044391959 kg
Portion_rice = 0.061068869 kg
Portion_sesamecake = 0.028329307 kg
Portion_sugars = 0.01109799 kg
Portion_tapbran = 0.079603353 kg
Portion_wheat = 0.16889385 kg
Portion_wheatbran = 0.055489949 kg
The total cost of this balanced diet is: 131.3 in Nigeria Currency 
